## Reproduce results of Scheme A

Paper: "Statistical supervised meta-ensemble algorithm for data linkage"

Kha Vo, Jitendra Jonnagaddala, Siaw-Teng Liaw

February 2019

Jounal of Biomedical Informatics

Paper: "Statistical supervised meta-ensemble algorithm for data linkage"

Kha Vo, Jitendra Jonnagaddala, Siaw-Teng Liaw

February 2019

Jounal of Biomedical Informatics


In [ ]:
import recordlinkage as rl, pandas as pd, numpy as np
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.utils import shuffle
from recordlinkage.preprocessing import phonetic
from numpy.random import choice
import collections, numpy
from IPython.display import clear_output
from sklearn.model_selection import train_test_split, KFold

In [ ]:
def generate_true_links(df): 
    # although the match_id column is included in the original df to imply the true links,
    # this function will create the true_link object identical to the true_links properties
    # of recordlinkage toolkit, in order to exploit "Compare.compute()" from that toolkit
    # in extract_function() for extracting features quicker.
    # This process should be deprecated in the future release of the UNSW toolkit.
    df["rec_id"] = df.index.values.tolist()
    indices_1 = []
    indices_2 = []
    processed = 0
    for match_id in df["match_id"].unique():
        if match_id != -1:    
            processed = processed + 1
            # print("In routine generate_true_links(), count =", processed)
            # clear_output(wait=True)
            linkages = df.loc[df['match_id'] == match_id]
            for j in range(len(linkages)-1):
                for k in range(j+1, len(linkages)):
                    indices_1 = indices_1 + [linkages.iloc[j]["rec_id"]]
                    indices_2 = indices_2 + [linkages.iloc[k]["rec_id"]]    
    links = pd.MultiIndex.from_arrays([indices_1,indices_2])
    return links

def generate_false_links(df, size):
    # A counterpart of generate_true_links(), with the purpose to generate random false pairs
    # for training. The number of false pairs in specified as "size".
    df["rec_id"] = df.index.values.tolist()
    indices_1 = []
    indices_2 = []
    unique_match_id = df["match_id"].unique()
    for j in range(size):
            false_pair_ids = choice(unique_match_id, 2)
            candidate_1_cluster = df.loc[df['match_id'] == false_pair_ids[0]]
            candidate_1 = candidate_1_cluster.iloc[choice(range(len(candidate_1_cluster)))]
            candidate_2_cluster = df.loc[df['match_id'] == false_pair_ids[1]]
            candidate_2 = candidate_2_cluster.iloc[choice(range(len(candidate_2_cluster)))]    
            indices_1 = indices_1 + [candidate_1["rec_id"]]
            indices_2 = indices_2 + [candidate_2["rec_id"]]  
    links = pd.MultiIndex.from_arrays([indices_1,indices_2])
    return links

def swap_fields_flag(f11, f12, f21, f22):
    return int((f11 == f22) and (f12 == f21))

def extract_features(df, links):
    c = rl.Compare()
    c.string('given_name', 'given_name', method='jarowinkler', label='y_name')
    c.string('given_name_soundex', 'given_name_soundex', method='jarowinkler', label='y_name_soundex')
    c.string('given_name_nysiis', 'given_name_nysiis', method='jarowinkler', label='y_name_nysiis')
    c.string('surname', 'surname', method='jarowinkler', label='y_surname')
    c.string('surname_soundex', 'surname_soundex', method='jarowinkler', label='y_surname_soundex')
    c.string('surname_nysiis', 'surname_nysiis', method='jarowinkler', label='y_surname_nysiis')
    c.exact('street_number', 'street_number', label='y_street_number')
    c.string('address_1', 'address_1', method='levenshtein', threshold=0.7, label='y_address1')
    c.string('address_2', 'address_2', method='levenshtein', threshold=0.7, label='y_address2')
    c.exact('postcode', 'postcode', label='y_postcode')
    c.exact('day', 'day', label='y_day')
    c.exact('month', 'month', label='y_month')
    c.exact('year', 'year', label='y_year')
        
    # Build features
    feature_vectors = c.compute(links, df, df)
    return feature_vectors

def generate_train_X_y(df):
    # This routine is to generate the feature vector X and the corresponding labels y
    # with exactly equal number of samples for both classes to train the classifier.
    pos = extract_features(df, train_true_links)
    train_false_links = generate_false_links(df, len(train_true_links))    
    neg = extract_features(df, train_false_links)
    X = pos.values.tolist() + neg.values.tolist()
    y = [1]*len(pos)+[0]*len(neg)
    X, y = shuffle(X, y, random_state=0)
    X = np.array(X)
    y = np.array(y)
    return X, y

def train_model(modeltype, modelparam, train_vectors, train_labels, modeltype_2):
    if modeltype == 'svm': # Support Vector Machine
        model = svm.SVC(C = modelparam, kernel = modeltype_2)
        model.fit(train_vectors, train_labels) 
    elif modeltype == 'lg': # Logistic Regression
        model = LogisticRegression(C=modelparam, penalty = modeltype_2,class_weight=None, dual=False, fit_intercept=True, 
                                   intercept_scaling=1, max_iter=5000, multi_class='ovr', 
                                   n_jobs=1, random_state=None)
        model.fit(train_vectors, train_labels)
    elif modeltype == 'nb': # Naive Bayes
        model = GaussianNB()
        model.fit(train_vectors, train_labels)
    elif modeltype == 'nn': # Neural Network
        model = MLPClassifier(solver='lbfgs', alpha=modelparam, hidden_layer_sizes=(256, ), 
                              activation = modeltype_2,random_state=None, batch_size='auto', 
                              learning_rate='constant',  learning_rate_init=0.001, 
                              power_t=0.5, max_iter=10000, shuffle=True, 
                              tol=0.0001, verbose=False, warm_start=False, momentum=0.9, 
                              nesterovs_momentum=True, early_stopping=False, 
                              validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
        model.fit(train_vectors, train_labels)
    return model

def classify(model, test_vectors):
    result = model.predict(test_vectors)
    return result

    
def evaluation(test_labels, result):
    true_pos = np.logical_and(test_labels, result)
    count_true_pos = np.sum(true_pos)
    true_neg = np.logical_and(np.logical_not(test_labels),np.logical_not(result))
    count_true_neg = np.sum(true_neg)
    false_pos = np.logical_and(np.logical_not(test_labels), result)
    count_false_pos = np.sum(false_pos)
    false_neg = np.logical_and(test_labels,np.logical_not(result))
    count_false_neg = np.sum(false_neg)
    precision = count_true_pos/(count_true_pos+count_false_pos)
    sensitivity = count_true_pos/(count_true_pos+count_false_neg) # sensitivity = recall
    confusion_matrix = [count_true_pos, count_false_pos, count_false_neg, count_true_neg]
    no_links_found = np.count_nonzero(result)
    no_false = count_false_pos + count_false_neg
    Fscore = 2*precision*sensitivity/(precision+sensitivity)
    metrics_result = {'no_false':no_false, 'confusion_matrix':confusion_matrix ,'precision':precision,
                     'sensitivity':sensitivity ,'no_links':no_links_found, 'F-score': Fscore}
    return metrics_result

def blocking_performance(candidates, true_links, df):
    count = 0
    for candi in candidates:
        if df.loc[candi[0]]["match_id"]==df.loc[candi[1]]["match_id"]:
            count = count + 1
    return count

In [ ]:
trainset = 'febrl3_UNSW'
testset = 'febrl4_UNSW'

In [ ]:
## TRAIN SET CONSTRUCTION

# Import
print("Import train set...")
df_train = pd.read_csv(trainset+".csv", index_col = "rec_id")
train_true_links = generate_true_links(df_train)
print("Train set size:", len(df_train), ", number of matched pairs: ", str(len(train_true_links)))

# Preprocess train set
df_train['postcode'] = df_train['postcode'].astype(str)
df_train['given_name_soundex'] = phonetic(df_train['given_name'], method='soundex')
df_train['given_name_nysiis'] = phonetic(df_train['given_name'], method='nysiis')
df_train['surname_soundex'] = phonetic(df_train['surname'], method='soundex')
df_train['surname_nysiis'] = phonetic(df_train['surname'], method='nysiis')

# Final train feature vectors and labels
X_train, y_train = generate_train_X_y(df_train)
print("Finished building X_train, y_train")

In [ ]:
# Blocking Criteria: declare non-match of all of the below fields disagree
# Import
print("Import test set...")
df_test = pd.read_csv(testset+".csv", index_col = "rec_id")
test_true_links = generate_true_links(df_test)
leng_test_true_links = len(test_true_links)
print("Test set size:", len(df_test), ", number of matched pairs: ", str(leng_test_true_links))

print("BLOCKING PERFORMANCE:")
blocking_fields = ["given_name", "surname", "postcode"]
all_candidate_pairs = []
for field in blocking_fields:
    block_indexer = rl.BlockIndex(on=field)
    candidates = block_indexer.index(df_test)
    detects = blocking_performance(candidates, test_true_links, df_test)
    all_candidate_pairs = candidates.union(all_candidate_pairs)
    print("Number of pairs of matched "+ field +": "+str(len(candidates)), ", detected ",
         detects,'/'+ str(leng_test_true_links) + " true matched pairs, missed " + 
          str(leng_test_true_links-detects) )
detects = blocking_performance(all_candidate_pairs, test_true_links, df_test)
print("Number of pairs of at least 1 field matched: " + str(len(all_candidate_pairs)), ", detected ",
     detects,'/'+ str(leng_test_true_links) + " true matched pairs, missed " + 
          str(leng_test_true_links-detects) )

In [ ]:
## TEST SET CONSTRUCTION

# Preprocess test set
print("Processing test set...")
print("Preprocess...")
df_test['postcode'] = df_test['postcode'].astype(str)
df_test['given_name_soundex'] = phonetic(df_test['given_name'], method='soundex')
df_test['given_name_nysiis'] = phonetic(df_test['given_name'], method='nysiis')
df_test['surname_soundex'] = phonetic(df_test['surname'], method='soundex')
df_test['surname_nysiis'] = phonetic(df_test['surname'], method='nysiis')

# Test feature vectors and labels construction
print("Extract feature vectors...")
df_X_test = extract_features(df_test, all_candidate_pairs)
vectors = df_X_test.values.tolist()
labels = [0]*len(vectors)
feature_index = df_X_test.index
for i in range(0, len(feature_index)):
    if df_test.loc[feature_index[i][0]]["match_id"]==df_test.loc[feature_index[i][1]]["match_id"]:
        labels[i] = 1
X_test, y_test = shuffle(vectors, labels, random_state=0)
X_test = np.array(X_test)
y_test = np.array(y_test)
print("Count labels of y_test:",collections.Counter(y_test))
print("Finished building X_test, y_test")

In [ ]:
## BASE LEARNERS CLASSIFICATION AND EVALUATION
# Choose model
print("BASE LEARNERS CLASSIFICATION PERFORMANCE:")
modeltype = 'lg' # choose between 'svm', 'lg', 'nn'
modeltype_2 = 'l2'  # 'linear' or 'rbf' for svm, 'l1' or 'l2' for lg, 'relu' or 'logistic' for nn
modelparam_range = [.001,.002,.005,.01,.02,.05,.1,.2,.5,1,5,10,20,50,100,200,500,1000,2000,5000] # C for svm, C for lg, alpha for NN
print("Model:",modeltype,", Param_1:",modeltype_2, ", tuning range:", modelparam_range)
precision = []
sensitivity = []
Fscore = []
nb_false = []

for modelparam in modelparam_range:
    md = train_model(modeltype, modelparam, X_train, y_train, modeltype_2)
    final_result = classify(md, X_test)
    final_eval = evaluation(y_test, final_result)
    precision += [final_eval['precision']]
    sensitivity += [final_eval['sensitivity']]
    Fscore += [final_eval['F-score']]
    nb_false  += [final_eval['no_false']]
    
print("No_false:",nb_false,"\n")
print("Precision:",precision,"\n")
print("Sensitivity:",sensitivity,"\n")
print("F-score:", Fscore,"\n")
print("")

In [ ]:
## ENSEMBLE CLASSIFICATION AND EVALUATION

print("BAGGING PERFORMANCE:\n")
modeltypes = ['svm', 'nn', 'lg'] 
modeltypes_2 = ['linear', 'relu', 'l2']
modelparams = [0.005, 100, 0.2]
nFold = 10
kf = KFold(n_splits=nFold)
model_raw_score = [0]*3
model_binary_score = [0]*3
model_i = 0
for model_i in range(3):
    modeltype = modeltypes[model_i]
    modeltype_2 = modeltypes_2[model_i]
    modelparam = modelparams[model_i]
    print(modeltype, "per fold:")
    iFold = 0
    result_fold = [0]*nFold
    final_eval_fold = [0]*nFold
    for train_index, valid_index in kf.split(X_train):
        X_train_fold = X_train[train_index]
        y_train_fold = y_train[train_index]
        md =  train_model(modeltype, modelparam, X_train_fold, y_train_fold, modeltype_2)
        result_fold[iFold] = classify(md, X_test)
        final_eval_fold[iFold] = evaluation(y_test, result_fold[iFold])
        print("Fold", str(iFold), final_eval_fold[iFold])
        iFold = iFold + 1
    bagging_raw_score = np.average(result_fold, axis=0)
    bagging_binary_score  = np.copy(bagging_raw_score)
    bagging_binary_score[bagging_binary_score > 0.5] = 1
    bagging_binary_score[bagging_binary_score <= 0.5] = 0
    bagging_eval = evaluation(y_test, bagging_binary_score)
    print(modeltype, "bagging:", bagging_eval)
    print('')
    model_raw_score[model_i] = bagging_raw_score
    model_binary_score[model_i] = bagging_binary_score

In [ ]:
thres = .99

print("STACKING PERFORMANCE:\n")
stack_raw_score = np.average(model_raw_score, axis=0)
stack_binary_score = np.copy(stack_raw_score)
stack_binary_score[stack_binary_score > thres] = 1
stack_binary_score[stack_binary_score <= thres] = 0
stacking_eval = evaluation(y_test, stack_binary_score)
print(stacking_eval)

## CS598 Project Code

The following contain our own code for replicating and validating the results of the original paper. The data sets and therefore data preprocessing constructs from the original paper are reused here rather than being reimplemented. Note that the implementation of both the paper's and our models are identical for both Schemes A and B; only hyperparameters differ.

The code is organized into sections:
1. Base Learner Bagging
     1. Support Vector Machine
     1. Neural Network
     1. Linear Regression
1. Stacking

Following our code is an "appendix" section which performs out-of-context validation on the reference implementation's base learners' performance using the same parameterization and data set.

### 1. Base Learners

In [ ]:
# Import the torch library which we'll use for our implementation
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Convert the numpy training and testing sets to torch.tensor for us with the PyTorch library
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).long()

#### 1.1. Support Vector Machine Base Learner

##### 1.1.1 SVM Model Implementation

The support vector machine implementation, FEBRLReproducerSVM, reproduces the results of the support vector machine base learner from the original paper. This model has two initialization parameters:
1. `num_features`: The number of features in the dataset; for the base dataset, this value is 13, but this value can differ if a dataset with fewer or additional features to be used.
1. `inverse_reg`: The hyperparameter that the paper explored via grid search to determine optimal inverse regularization strength for the optimizer.

In [ ]:
# CS598 Project Code / Support Vector Machine / Implementation

# For SVM, shift our data from its native range of [0.0, 1.0] to [-1.0, 1.0]
X_train_tensor_svm = (X_train_tensor * 2) - 1
y_train_tensor_svm = (y_train_tensor * 2) - 1
X_test_tensor_svm = (X_test_tensor * 2) - 1
y_test_tensor_svm = (y_test_tensor * 2) - 1

class FEBRLReproducerSVM(nn.Module):
    def __init__(self, num_features, inverse_reg=0.0):
        # Create the our PyTorch support vector machine model
        super(FEBRLReproducerSVM, self).__init__()

        # STEP 1
        # Specify parameters for our PyTorch SVM model based upon the analogous
        # parameters used by the original paper's sklearn SVC

        # PyTorch SVM (nn) concept                       Analogous sklearn SVC parameter
        # ------------------------                      -------------------------------
        self.inverse_reg = inverse_reg                  # C (inverse of the regularization strength)
        #                                               # kernel (original paper uses linear, as do we)

        # STEP 2
        # Define the layers for our lr model
        self.num_input_features = num_features

        self.fc1 = nn.Linear(in_features=self.num_input_features, out_features=1, bias=False)

        # STEP 3
        # Define the criteria and optimizer
        self.num_max_epochs = 1000
        self.criterion = nn.HingeEmbeddingLoss()
        self.optimizer = torch.optim.SGD(self.parameters(),
            lr = 0.001,
            weight_decay=inverse_reg)

    def forward(self, x):
        # Perform a forward pass on the nn; it is not recommended to call this
        # function directly, but to instead call fit(...) or predict(...) so that model's
        # mode is correctly set automatically
        x = self.fc1(x)

        return torch.squeeze(x)

    def fit(self, X_train, y_train):
        # Train the SVM with the specified parameters; analogous to sklearn's
        # SVC.fit(...) method
        self.train()

        loss_previous_epoch = 1.0
        loss_consecutive_epochs_minimal = 0

        for epoch_i in np.arange(self.num_max_epochs):
            loss = None
            kfold = KFold(n_splits=10, shuffle=True, random_state=12345)

            for train_indicies, _ in kfold.split(X_train):
                self.optimizer.zero_grad()
                output = self.forward(X_train[train_indicies])
                output *= -1
                loss = self.criterion(output, y_train[train_indicies])
                loss.backward()
                self.optimizer.step()

            # Determine if criteria for early training termination is satisfied
            if (np.abs(loss_previous_epoch - loss.item())) <= 0.0001:
                loss_consecutive_epochs_minimal = loss_consecutive_epochs_minimal + 1

                if(loss_consecutive_epochs_minimal == 50):
                    break
            else:
                loss_consecutive_epochs_minimal = 0

            loss_previous_epoch = loss.item()

    def predict(self, X_test):
        # Test the nn with the specified parameters; analogous to sklearn's
        # SVC.predict(...) method
        self.eval()
        return self.forward(X_test)

frs_inverse_reg_range = [.001,.002,.005,.01,.02,.05,.1,.2,.5,1,5,10,20,50,100,200,500,1000,2000,5000] 
frs_inverse_reg_optimal = 1000  # Determined through search

##### 1.1.2. Support Vector Machine Hyperparameter Search

Grid search is performed on the set of candidate hyperparameters from the original paper using our model. Some hyperparameters may take a long time to test if they do not cause the model's loss to converge early (or at all).

In [ ]:
# CS598 Project Code / Support Vector Machine / Hyperparameter Search

# Perform SVM base learner evaluation using the hyperparameter search range provided by the original paper
for inverse_reg in frs_inverse_reg_range:
    # Create an instance of the SVM
    febrl_reproducer_svm = FEBRLReproducerSVM(num_features=X_train_tensor.shape[1], inverse_reg=inverse_reg)

    # Train the model
    febrl_reproducer_svm.fit(X_train_tensor_svm, y_train_tensor_svm)

    # Test the model
    frs_output = febrl_reproducer_svm.predict(X_test_tensor_svm).detach()

    y_pred = np.asarray([1 if element > 0 else 0 for element in frs_output])

    print("weight_decay = {}: {}".format(inverse_reg, evaluation(y_test, y_pred)))

##### 1.1.3. Support Vector Machine Training, Evaluation, and Bagging

The reference implementation uses a 10-split k-fold as their bootstrapping technique. We will do the same here to ensure that our implementation is trained with the same data as the reference implementation. After each base learner is trained, it is evaluated with the test data set. After all base learners have evaluated the test data set, their outputs are passed through the bagging classifier.

In [ ]:
# CS598 Project Code / Support Vector Machine / Training, Evaluation, and Bagging

# Perform bagging across 10 models
frs_kfold_count = 10
frs_kfold = KFold(n_splits=frs_kfold_count, shuffle=True, random_state=12345)
frs_kfold_i = 0

frs_results = [0] * frs_kfold_count

for train_indicies, _ in frs_kfold.split(X_train):
    # Create an instance of the feed-forward neural network
    febrl_reproducer_svm = FEBRLReproducerSVM(num_features=X_train_tensor.shape[1], inverse_reg=frs_inverse_reg_optimal)

    # Train the model
    febrl_reproducer_svm.fit(X_train_tensor_svm[train_indicies], y_train_tensor_svm[train_indicies])

    # Test the model
    frs_results[frs_kfold_i] = febrl_reproducer_svm.predict(X_test_tensor_svm).detach().numpy()

    # Print the results of the current base learner for convenience
    y_pred = np.asarray([1 if element > 0 else 0 for element in frs_results[frs_kfold_i]])
    print("Execution {}: {}".format(frs_kfold_i, evaluation(y_test, y_pred)))

    frs_kfold_i = frs_kfold_i + 1

frs_bagging_raw_score = np.average(frs_results, axis=0)
frs_bagging_binary_score = np.copy(frs_bagging_raw_score)
frs_bagging_binary_score[frs_bagging_binary_score > 0] = 1
frs_bagging_binary_score[frs_bagging_binary_score <= 0] = 0
frs_bagging_evaluation = evaluation(y_test, frs_bagging_binary_score)
print("SVM bagging: {}".format(frs_bagging_evaluation))

#### 1.2 Neural Network Base Learner

##### 1.2.1. Neural Network Model Implementation

The neural network implementation, FEBRLReproducerNN, reproduces the results of the neural network base learner from the original paper. This model has two initialization parameters:
1. `num_features`: The number of features in the dataset; for the base dataset, this value is 13, but this value can differ if a dataset with fewer or additional features to be used.
1. `weight_decay`: The hyperparameter that the paper explored via grid search to determine optimal weight decay for the optimizer.

In [ ]:
# CS598 Project Code / Neural Network Model / Implementation

class FEBRLReproducerNN(nn.Module):
    def __init__(self, num_features, weight_decay=0.0):
        # Create the our PyTorch nn model
        super(FEBRLReproducerNN, self).__init__()

        # STEP 1
        # Specify parameters for our PyTorch nn model based upon the analogous
        # parameters used by the original paper's sklearn MLPClassifier

        # PyTorch nn concept                            Analogous sklearn MLPClassifier parameter
        # ------------------                            -----------------------------------------
        self.optimizer = None                           # solver (optimizer; original paper uses LBFGS, but we will use SGD (defined later) due to PyTorch-sklearn differences)
        self.optimizer_weight_decay = weight_decay      # alpha (L2 penalty/regularization term)
        self.num_hidden_layer_nodes = 64                # hidden_layer_sizes (tuple of hidden layer nodes; original paper uses 256, see "[2]" below in Step 2)
        self.activation = F.relu                        # activation (activation function)
        self.random_state = 12345                       # random_state (static, random state for reproducibility)
        #                                               # batch_size (minibatch size; unused in our model)
        #                                               # learning_rate (tells the model to use the provided initial learning rate; n/a to our model)
        self.optimizer_learning_rate_init = 0.05        # learning_rate_init (initial learning rate; original paper uses 0.001; see "[1]" below)
        self.optimizer_dampening = 0.0                  # power_t (dampening)
        self.num_max_epochs = 3000                      # max_iter (maximum number of epochs when using stochastic optimizers)
        self.shuffle = True                             # shuffle (shuffle samples in each iteration)
        self.tolerance = 0.0001                         # tol (optimization tolorance; early training termination)
        #                                               # verbose (print model progress debug messages to console; specified by unused by original paper)
        #                                               # warm_start (initialize the model with the results of previous executions; specified but unused by original paper)
        self.optimizer_momentum = 0.9                   # momentum (optimizer momentum)
        self.use_nesterov_momentum = True               # nesterovs_momentum (use Nesterov's momentum in the optimizer)
        #                                               # early_stopping (terminate early when validation is not improving; 'False' in original paper)
        #                                               # validation_fraction (validation data set criteria for early stopping; specified by unused by original paper)
        #                                               # beta_1 (parameter for Adam optimizer; specified but unused by original paper)
        #                                               # beta_2 (parameter for Adam optimizer; specified but unused by original paper)
        #                                               # epsilon (parameter for Adam optimizer; specified but unused by original paper)

        # [1] The original authors' model used a learning rate of 0.001, however, we discovered that with our
        #     model's adjusted hyperparameters, a slightly larger learning rate resulted in a measurably higher
        #     sensitivity score of approximately 3%

        # STEP 2
        # Define the layers for our nn model
        # [2] The original authors' model uses a single hidden layer of 256 nodes, but we found this decision
        #     to be suboptimal; we determined a much more optimal architecture using two hidden layers, with
        #     64 and 16 nodes, respectively
        self.num_input_features = num_features

        self.fc1 = nn.Linear(in_features=self.num_input_features, out_features=self.num_hidden_layer_nodes, bias=False)
        self.fc2 = nn.Linear(in_features=64, out_features=16, bias=False)
        self.fc3 = nn.Linear(in_features=16, out_features=1, bias=False)

        # STEP 3
        # Define the criteria and optimizer
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.SGD(self.parameters(),
            lr=self.optimizer_learning_rate_init,
            weight_decay=self.optimizer_weight_decay,
            momentum=self.optimizer_momentum,
            dampening=self.optimizer_dampening,
            nesterov=self.use_nesterov_momentum)

    def forward(self, x):
        # Perform a forward pass on the nn; it is not recommended to call this
        # function directly, but to instead call fit(...) or predict(...) so that model's
        # mode is correctly set automatically
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)

        return torch.squeeze(x)

    def fit(self, X_train, y_train):
        # Train the nn with the specified parameters; analogous to sklearn's
        # MLPClassifier.fit(...) method
        self.train()
    
        loss_previous_epoch = 1.0
        loss_consecutive_epochs_minimal = 0

        for epoch_i in np.arange(self.num_max_epochs):
            loss = None
            kfold = KFold(n_splits=10, shuffle=self.shuffle, random_state=self.random_state)

            for train_indicies, _ in kfold.split(X_train):
                self.optimizer.zero_grad()
                output = self.forward(X_train[train_indicies])
                loss = self.criterion(output, y_train[train_indicies])
                loss.backward()
                self.optimizer.step()

            # Determine if criteria for early training termination is satisfied
            if (loss_previous_epoch - loss.item()) <= self.tolerance:
                loss_consecutive_epochs_minimal = loss_consecutive_epochs_minimal + 1

                if(loss_consecutive_epochs_minimal == 50):
                    break
            else:
                loss_consecutive_epochs_minimal = 0

            loss_previous_epoch = loss.item()

    def predict(self, X_test):
        # Test the nn with the specified parameters; analogous to sklearn's
        # MLPClassifier.predict(...) method
        self.eval()
        return self.forward(X_test)

frn_weight_decay_range = [.001,.002,.005,.01,.02,.05,.1,.2,.5,1,5,10,20,50,100,200,500,1000,2000,5000]
frn_weight_decay_optimal = 0.5  # Determined through search

##### 1.2.2. Neural Network Hyperparameter Search

Grid search is performed on the set of candidate hyperparameters from the original paper using our model. Some hyperparameters may take a long time to test if they do not cause the model's loss to converge early (or at all).

In [ ]:
# CS598 Project Code / Neural Network Model / Hyperparameter Search

# Perform nn base learner evaluation using the hyperparameter search range provided by the original paper
for weight_decay in frn_weight_decay_range:
    # Create an instance of the feed-forward neural network
    febrl_reproducer_nn = FEBRLReproducerNN(num_features=X_train_tensor.shape[1], weight_decay=weight_decay)

    # Train the model
    febrl_reproducer_nn.fit(X_train_tensor, y_train_tensor)

    # Test the model
    frn_output = febrl_reproducer_nn.predict(X_test_tensor).detach()

    y_pred = np.asarray([1 if element > 0.5 else 0 for element in frn_output])

    print("weight_decay = {}: {}".format(weight_decay, evaluation(y_test, y_pred)))

##### 1.2.3. Neural Network Training, Evaluation, and Bagging

The reference implementation uses a 10-split k-fold as their bootstrapping technique. We will do the same here to ensure that our implementation is trained with the same data as the reference implementation. After each base learner is trained, it is evaluated with the test data set. After all base learners have evaluated the test data set, their outputs are passed through the bagging classifier.

In [ ]:
# CS598 Project Code / Neural Network Model / Training, Evaluation, and Bagging

# Perform bagging across 10 models
frn_kfold_count = 10
frn_kfold = KFold(n_splits=frn_kfold_count, shuffle=True, random_state=12345)
frn_kfold_i = 0

frn_results = [0] * frn_kfold_count

for train_indicies, _ in frn_kfold.split(X_train):
    # Create an instance of the feed-forward neural network
    febrl_reproducer_nn = FEBRLReproducerNN(num_features=X_train_tensor.shape[1], weight_decay=frn_weight_decay_optimal)

    # Train the model
    febrl_reproducer_nn.fit(X_train_tensor[train_indicies], y_train_tensor[train_indicies])

    # Test the model
    frn_results[frn_kfold_i] = febrl_reproducer_nn.predict(X_test_tensor).detach().numpy()

    # Print the results of the current base learner for convenience
    y_pred = np.asarray([1 if element > 0.5 else 0 for element in frn_results[frn_kfold_i]])
    print("Execution {}: {}".format(frn_kfold_i, evaluation(y_test, y_pred)))

    frn_kfold_i = frn_kfold_i + 1

frn_bagging_raw_score = np.average(frn_results, axis=0)
frn_bagging_binary_score = np.copy(frn_bagging_raw_score)
frn_bagging_binary_score[frn_bagging_binary_score > 0.5] = 1
frn_bagging_binary_score[frn_bagging_binary_score <= 0.5] = 0
frn_bagging_evaluation = evaluation(y_test, frn_bagging_binary_score)
print("Neural Network bagging: {}".format(frn_bagging_evaluation))

#### 1.3. Logistic Regression Base Learner

##### 1.3.1 Logistic Regression Model Implementation

The logistic regression implementation, FEBRLReproducerLR, reproduces the results of the logistic regression base learner from the original paper. This model has two initialization parameters:
1. `num_features`: The number of features in the dataset; for the base dataset, this value is 13, but this value can differ if a dataset with fewer or additional features to be used.
1. `inverse_reg`: The hyperparameter that the paper explored via grid search to determine optimal inverse regularization strength for the optimizer.

In [ ]:
# CS598 Project Code / Logistic Regression / Implementation

class FEBRLReproducerLR(nn.Module):
    def __init__(self, num_features, inverse_reg=0.0):
        # Create the our PyTorch logistic regression model
        super(FEBRLReproducerLR, self).__init__()

        # STEP 1
        # Specify parameters for our PyTorch LR model based upon the analogous
        # parameters used by the original paper's sklearn LogisticRegression

        # PyTorch LR (nn) concept                       Analogous sklearn LogisticRegression parameter
        # -----------------------                       ----------------------------------------------
        self.inverse_reg = inverse_reg                  # C (inverse of the regularization strength)
        #                                               # penalty (original paper uses L2)
        #                                               # dual (specifies dual formulation; specified but unused by the original paper)
        self.use_bias = True                            # fit_intercept (specified if bias should be added to decision function; original paper specified this as true)
        #                                               # intercept_scaling (intercept scaling, the original paper specifies this as 1)
        self.num_max_epochs = 10000                     # max_iter (maximum number of epochs when using stochastic optimizers)
        #                                               # multi_class (specifies class of problem; ours is a binary classification problem)
        #                                               # n_jobs (the number of CPU cores used for parallelization; specified but unused by the original paper)
        self.random_state = 12345                       # random_state (static, random state for reproducibility)

        # STEP 2
        # Define the layers for our lr model
        self.num_input_features = num_features

        self.fc1 = nn.Linear(in_features=self.num_input_features, out_features=1, bias=self.use_bias)

        # STEP 3
        # Define the criteria and optimizer
        self.criterion = nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.SGD(self.parameters(),
            lr = 0.05,
            weight_decay=self.inverse_reg)

    def forward(self, x):
        # Perform a forward pass on the nn; it is not recommended to call this
        # function directly, but to instead call fit(...) or predict(...) so that model's
        # mode is correctly set automatically
        x = self.fc1(x)

        return torch.squeeze(x)

    def fit(self, X_train, y_train):
        # Train the nn with the specified parameters; analogous to sklearn's
        # LogisticRegression.fit(...) method
        self.train()

        loss_previous_epoch = 1.0
        loss_consecutive_epochs_minimal = 0

        for epoch_i in np.arange(self.num_max_epochs):
            loss = None
            kfold = KFold(n_splits=10, shuffle=True, random_state=self.random_state)

            for train_indicies, _ in kfold.split(X_train):
                self.optimizer.zero_grad()
                output = self.forward(X_train[train_indicies])
                loss = self.criterion(output, y_train[train_indicies])
                loss.backward()
                self.optimizer.step()

            # Determine if criteria for early training termination is satisfied
            if (loss_previous_epoch - loss.item()) <= 0.0001:
                loss_consecutive_epochs_minimal = loss_consecutive_epochs_minimal + 1

                if(loss_consecutive_epochs_minimal == 50):
                    break
            else:
                loss_consecutive_epochs_minimal = 0

            loss_previous_epoch = loss.item()

    def predict(self, X_test):
        # Test the LR with the specified parameters; analogous to sklearn's
        # LogisticRegression.predict(...) method
        self.eval()
        return self.forward(X_test)

frl_inverse_reg_range = [.001,.002,.005,.01,.02,.05,.1,.2,.5,1,5,10,20,50,100,200,500,1000,2000,5000] 
frl_inverse_reg_optimal = 0.5  # Determined through search

##### 1.3.2 Logistic Regression Hyperparameter Search

Grid search is performed on the set of candidate hyperparameters from the original paper using our model. Some hyperparameters may take a long time to test if they do not cause the model's loss to converge early (or at all).

In [ ]:
# CS598 Project Code / Logistic Regression Model / Hyperparameter Search

# Perform logistic regression base learner evaluation using the hyperparameter search range provided by the original paper
for inverse_reg in frl_inverse_reg_range:
    # Create an instance of the logistic regression model
    febrl_reproducer_lr = FEBRLReproducerLR(num_features=X_train_tensor.shape[1], inverse_reg=inverse_reg)

    # Train the model
    febrl_reproducer_lr.fit(X_train_tensor, y_train_tensor)

    # Test the model
    frl_output = febrl_reproducer_lr.predict(X_test_tensor).detach()

    y_pred = np.asarray([1 if element > 0.5 else 0 for element in frl_output])

    print("inverse_reg = {}: {}".format(inverse_reg, evaluation(y_test, y_pred)))

##### 1.3.3 Logistic Regression Training, Evaluation, and Bagging

The reference implementation uses a 10-split k-fold as their bootstrapping technique. We will do the same here to ensure that our implementation is trained with the same data as the reference implementation. After each base learner is trained, it is evaluated with the test data set. After all base learners have evaluated the test data set, their outputs are passed through the bagging classifier.

In [ ]:
# CS598 Project Code / Logistic Regression Model / Training, Evaluation, and Bagging

# Perform bagging across 10 models
frl_kfold_count = 10
frl_kfold = KFold(n_splits=frl_kfold_count, shuffle=True, random_state=12345)
frl_kfold_i = 0

frl_results = [0] * frl_kfold_count

for train_indicies, _ in frl_kfold.split(X_train):
    # Create an instance of the feed-forward neural network
    febrl_reproducer_nn = FEBRLReproducerLR(num_features=X_train_tensor.shape[1], inverse_reg=frl_inverse_reg_optimal)

    # Train the model
    febrl_reproducer_nn.fit(X_train_tensor[train_indicies], y_train_tensor[train_indicies])

    # Test the model
    frl_results[frl_kfold_i] = febrl_reproducer_nn.predict(X_test_tensor).detach().numpy()

    # Print the results of the current base learner for convenience
    y_pred = np.asarray([1 if element > 0.5 else 0 for element in frl_results[frl_kfold_i]])
    print("Execution {}: {}".format(frl_kfold_i, evaluation(y_test, y_pred)))

    frl_kfold_i = frl_kfold_i + 1

frl_bagging_raw_score = np.average(frl_results, axis=0)
frl_bagging_binary_score = np.copy(frl_bagging_raw_score)
frl_bagging_binary_score[frl_bagging_binary_score > 0.5] = 1
frl_bagging_binary_score[frl_bagging_binary_score <= 0.5] = 0
frl_bagging_evaluation = evaluation(y_test, frl_bagging_binary_score)
print("Logistic Regression bagging: {}".format(frl_bagging_evaluation))

### 2. Stacking

The bagged base learners are stacked to produce a the model's final prediction.

In [ ]:
# CS598 Project Code / Stacking / Ensemble Prediction

fr_stacking_threshold = 0.99

fr_stacking_binary_score = np.average([frs_bagging_binary_score, frn_bagging_binary_score, frl_bagging_binary_score], axis=0)
fr_stacking_binary_score[fr_stacking_binary_score > fr_stacking_threshold] = 1
fr_stacking_binary_score[fr_stacking_binary_score <= fr_stacking_threshold] = 0
fr_stacking_evaluation = evaluation(y_test, fr_stacking_binary_score)
print("Ensemble bagging-stacking: {}".format(fr_stacking_evaluation))

### Appendix: Reference Implementation Validation

#### Base Learners

Each of the following cells executes one of the three base learner models from the sklearn library used by the reference implementation with identical parameterization and data set as a sanity check to validate the paper's models' results using the same data set used by the reference implementation above.

In [ ]:
# CS598 Project Code / Reference Implementation Validation (Neural Network)

# Sanity check: Create, train, and test a new instance of an sklearn MLPClassifier from scratch using the original
# paper's parameters to confirm that results are reproducible and absent any unexpected/hidden dependencies
nn_validation_model = MLPClassifier(solver='lbfgs', alpha=100, hidden_layer_sizes=(256, ), 
                              activation = 'relu',random_state=None, batch_size='auto', 
                              learning_rate='constant',  learning_rate_init=0.001, 
                              power_t=0.5, max_iter=10000, shuffle=True, 
                              tol=0.0001, verbose=False, warm_start=False, momentum=0.9, 
                              nesterovs_momentum=True, early_stopping=False, 
                              validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
nn_validation_model.fit(X_train, y_train)

nn_validation_model_results = classify(nn_validation_model, X_test)

print("Original paper nn: {}".format(evaluation(y_test, nn_validation_model_results)))

In [ ]:
# CS598 Project Code / Reference Implementation Validation (Logistic Regression)

# Sanity check: Create, train, and test a new instance of an sklearn LogisticRegression from scratch using the original
# paper's parameters to confirm that results are reproducible and absent any unexpected/hidden dependencies
lg_validation_model = LogisticRegression(C=0.2, penalty = 'l2',class_weight=None, dual=False, fit_intercept=True, 
                                   intercept_scaling=1, max_iter=5000, multi_class='ovr', 
                                   n_jobs=1, random_state=None)
lg_validation_model.fit(X_train, y_train)

lg_validation_model_results = classify(lg_validation_model, X_test)

print("Original paper logistic regression: {}".format(evaluation(y_test, lg_validation_model_results)))

In [ ]:
# CS598 Project Code / Reference Implementation Validation (Support Vector Machine)

# Sanity check: Create, train, and test a new instance of an sklearn SVC from scratch using the original
# paper's parameters to confirm that results are reproducible and absent any unexpected/hidden dependencies
svm_validation_model = svm.SVC(C = 0.005, kernel = 'linear')
svm_validation_model.fit(X_train, y_train)

svm_validation_model_results = classify(svm_validation_model, X_test)

print("Original paper support vector machine: {}".format(evaluation(y_test, svm_validation_model_results)))

In [ ]:
np.save("a_x_train", X_train)
np.save("a_y_train", y_train)
np.save("a_x_test", X_test)
np.save("a_y_test", y_test)